# BERT

## 1. 샘플 코드 체험해보기 

In [5]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [6]:
# 탐색 1. Tokenizer
# Tokenizer -> ?? 텍스트를 넣으면 그 텍스트를 어떤 숫자 tensor로 바꿔주는 역할
print(type(encoded_input))
print(encoded_input.keys())
print(encoded_input["input_ids"])
print(encoded_input["token_type_ids"])
print(encoded_input["attention_mask"])

<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
tensor([[  101, 72337, 72654, 10911, 10155, 11178, 15541, 13028,   112,   172,
         11850,   119,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [7]:
# 탐색 2. model에 넣어줘야 하는 값
# model(**encoded_input)
# 입력값이 딕셔너리여야 하며 그 키는 input_ids, token_type_ids, attention_mask
# 즉, 모델에 대한 입력값은 toknizer의 결과 출력값이다.

In [8]:
# 3. 탐색 3. 모델 마지막 출력 바꿔야 한다.
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [9]:
print(output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.2524, -0.5321,  0.4496,  ...,  1.1428, -0.6238, -0.0745],
         [ 0.5374,  0.0701,  0.4880,  ...,  1.0800, -0.5560, -0.5564],
         [ 0.4686, -0.3286,  0.4782,  ...,  1.1656, -0.7389, -0.3920],
         ...,
         [ 0.4822, -1.0822,  0.9026,  ...,  1.8029, -1.1343, -0.1034],
         [ 0.1283, -0.6550,  0.3723,  ...,  0.7817, -0.9173, -0.0401],
         [ 0.1645, -0.5239,  0.6638,  ...,  0.7964, -0.7326, -0.2642]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.3479, -0.0466,  0.4564, -0.2464, -0.0563,  0.5865,  0.4807,  0.2475,
         -0.5663,  0.4260,  0.0136, -0.3356, -0.3281,  0.0391,  0.3874, -0.4077,
          0.8111,  0.2171,  0.4523, -0.5184, -0.9998, -0.5465, -0.1472, -0.5278,
         -0.5858,  0.3390, -0.4160,  0.3991,  0.4012, -0.4298,  0.1618, -0.9998,
          0.7481,  0.7902,  0.3325, -0.4037,  0.1348,  0.2374,  0.2554, -0.1603,
         -0.3249,  0.0911, -0.49

## 2. 데이터 학습하기

### 1) 데이터 불러오기

In [10]:
import pandas as pd

data = pd.read_csv("./배달의민족댓글2.csv", index_col=0).dropna().reset_index(drop=True)
data.head()

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3


In [11]:
# 3이상이면 긍정 1, 그 외에는 0
import numpy as np

data["label"] = np.where(data["score"] >=3, 1, 0)
data.head()

,text,score,label
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4,1
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5,1
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1,0
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2,0
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3,1


### 2) 데이터 전처리

In [12]:
import re 
data["review"] = data["text"].apply(lambda x: re.sub("[^0-9a-zA-Z가-힣\s+]", "", x))
data.head()

,text,score,label,review
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4,1,배달의민족 주문시 리뷰를 자주 참고하는 편입니다 한가지 건의사항이 있다면 최신순별점...
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5,1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요 분명 이 가게에 시킨 기억...
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1,0,검색 화면에서 전체배달포장 탭 중 배달 탭을 스크롤 내리면서 볼 때 아래로 스크롤하...
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2,0,배달팁 낮은 순으로 정렬하면 04000원 이런식으로 된 가게가 가장 위로 올라옵니다...
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3,1,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다 배민 어플 실행시 업데...


In [13]:
data.shape

(1000, 4)

### 3) 데이터 분할하기

In [14]:
from sklearn.model_selection import train_test_split 

trainX, testX, trainY, testY = train_test_split(
    data["review"],
    data["label"],
    test_size=0.2,
    random_state=42
)

print(f"TrainX: {trainX.shape}, TrainY: {trainY.shape}")
print(f"TestX: {testX.shape}, TestY: {testY.shape}")

TrainX: (800,), TrainY: (800,)
TestX: (200,), TestY: (200,)


In [15]:
trainX, valX, trainY, valY = train_test_split(
    trainX, trainY, test_size=0.1, random_state=42
)

print(f"TrainX: {trainX.shape}, TrainY: {trainY.shape}")
print(f"ValX: {valX.shape}, ValY: {valY.shape}")
print(f"TestX: {testX.shape}, TestY: {testY.shape}")

TrainX: (720,), TrainY: (720,)
ValX: (80,), ValY: (80,)
TestX: (200,), TestY: (200,)


In [16]:
trainX.head()

940    주문이 누락됬으면 누락됬다고 알려줘야죠 배달 예정 시간이 지나고 30분이나 지나서 ...
547     잘되면 감사하고 더 많은 서비스를 줘야지 고객들 호구로보다가 코로나랑 배달회사랑 ...
900    항상 잘 사용하고있습니다 배민에서 오는 알림 내역을 다시 볼 수 있는 목록도 있었으...
603    배달팁이 기본 4000원이 넘어갑니다 장난하나요 7000원인 곳도 있어요 물가 올라...
348    전에 먹었던 메뉴를 그대로 재주문 할 수 있는 기능이 있으면 좋겠어요 혹시 있나 찾...
Name: review, dtype: object

### 4) 학습데이터 만들기

In [17]:
# CustomDataset Class를 만들어야 한다.
# __len__ : 데이터의 개수를 반환한다.
# __getitem__ : 학습 데이터 1개 딕셔너리를 반환한다.
## tokenizer()의 출력(딕셔너리) + label키를 추가


#### 테스트1 - 샘플코드 그대로

In [18]:
import torch 
from torch.utils.data import Dataset, DataLoader 

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts.tolist() 
        self.labels = labels.tolist()
        self.tokenizer = tokenizer

    def tokenize(self, text):
        encoded_output = self.tokenizer(
            text, 
            return_tensors='pt'
        )
        return encoded_output
            
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 데이터 가져오기
        text = self.texts[idx]
        label = self.labels[idx]

        # 토크나이징
        encoding = self.tokenize(text)

        # 라벨 추가
        # encoding["label"] = torch.tensor(label, dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "token_type_ids": encoding["token_type_ids"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = CustomDataset(trainX, trainY, tokenizer)
print(len(train_dataset))
# print(train_dataset[0])
print(train_dataset[0]["input_ids"].shape)
print(train_dataset[0]["attention_mask"].shape)
print(train_dataset[0]["token_type_ids"].shape)
print(train_dataset[1]["input_ids"].shape)
print(train_dataset[1]["attention_mask"].shape)
print(train_dataset[1]["token_type_ids"].shape)
print(train_dataset[2]["input_ids"].shape)
print(train_dataset[2]["attention_mask"].shape)
print(train_dataset[2]["token_type_ids"].shape)

720
torch.Size([74])
torch.Size([74])
torch.Size([74])
torch.Size([42])
torch.Size([42])
torch.Size([42])
torch.Size([65])
torch.Size([65])
torch.Size([65])


#### 테스트 2 - flatten이 없을 시

In [20]:
import torch 
from torch.utils.data import Dataset, DataLoader 

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts.tolist() 
        self.labels = labels.tolist()
        self.tokenizer = tokenizer

    def tokenize(self, text):
        encoded_output = self.tokenizer(
            text, 
            max_length = 128,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return encoded_output
            
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 데이터 가져오기
        text = self.texts[idx]
        label = self.labels[idx]

        # 토크나이징
        encoding = self.tokenize(text)

        # 라벨 추가
        # encoding["label"] = torch.tensor(label, dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"],
            "attention_mask": encoding["attention_mask"],
            "token_type_ids": encoding["token_type_ids"],
            "label": torch.tensor(label, dtype=torch.long)
        }

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = CustomDataset(trainX, trainY, tokenizer)
print(len(train_dataset))
print(train_dataset[0]["input_ids"])
# print(train_dataset[0]["attention_mask"].shape)
# print(train_dataset[0]["token_type_ids"].shape)
print(train_dataset[1]["input_ids"])
# print(train_dataset[1]["attention_mask"].shape)
# print(train_dataset[1]["token_type_ids"].shape)
print(train_dataset[2]["input_ids"])
# print(train_dataset[2]["attention_mask"].shape)
# print(train_dataset[2]["token_type_ids"].shape)

720
tensor([[   101,   9689,  25934,  10739,    100,    100,   9524,  26737, 119221,
          21711, 119217,   9330,  89851,   9576,  16605,   9485,  96618,   9706,
          16439,  11664,  10244,  37712,  43739,   9706,  16439,  12424,   9258,
          34907,  10530,   9665,  18227,  35506,  25503,   9524,  14153, 118800,
          77884,  48549,   9995, 118992, 118965,  11018,   8898,  16605,  12092,
           9357, 119199,  12453,  48549,  62849,  15303,   9580,  46520,  12030,
         119219,   9524, 119118,  41850,   9435,  35465,  43739,  80956,   9641,
          10739,  90587,   9668,  34907,   9536,  10622,   9521,   9511,  14153,
            100,    102,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,

In [22]:
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for x in train_loader:
    print(x["input_ids"].shape)
    print(x["attention_mask"].shape)
    print(x["token_type_ids"].shape)
    break

torch.Size([8, 1, 128])
torch.Size([8, 1, 128])
torch.Size([8, 1, 128])


#### ⭐ (확정)테스트 3 

In [23]:
import torch 
from torch.utils.data import Dataset, DataLoader 

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.tolist() 
        self.labels = labels.tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def tokenize(self, text):
        encoded_output = self.tokenizer(
            text, 
            max_length = self.max_length,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return encoded_output
            
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 데이터 가져오기
        text = self.texts[idx]
        label = self.labels[idx]

        # 토크나이징
        encoding = self.tokenize(text)

        # 라벨 추가
        # encoding["label"] = torch.tensor(label, dtype=torch.long)

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "token_type_ids": encoding["token_type_ids"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_dataset = CustomDataset(trainX, trainY, tokenizer, max_length=128)
print(len(train_dataset))
# print(train_dataset[0])
print(train_dataset[0]["input_ids"])
print(train_dataset[0]["attention_mask"])
print(train_dataset[0]["token_type_ids"])

720
tensor([   101,   9689,  25934,  10739,    100,    100,   9524,  26737, 119221,
         21711, 119217,   9330,  89851,   9576,  16605,   9485,  96618,   9706,
         16439,  11664,  10244,  37712,  43739,   9706,  16439,  12424,   9258,
         34907,  10530,   9665,  18227,  35506,  25503,   9524,  14153, 118800,
         77884,  48549,   9995, 118992, 118965,  11018,   8898,  16605,  12092,
          9357, 119199,  12453,  48549,  62849,  15303,   9580,  46520,  12030,
        119219,   9524, 119118,  41850,   9435,  35465,  43739,  80956,   9641,
         10739,  90587,   9668,  34907,   9536,  10622,   9521,   9511,  14153,
           100,    102,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,     

In [25]:
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
for x in train_loader:
    print(x["input_ids"].shape)
    print(x["attention_mask"].shape)
    print(x["token_type_ids"].shape)
    break

torch.Size([8, 128])
torch.Size([8, 128])
torch.Size([8, 128])


#### 최종 정리

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 128 
train_dataset = CustomDataset(trainX, trainY, tokenizer, max_length)
val_dataset = CustomDataset(valX, valY, tokenizer, max_length)
test_dataset = CustomDataset(testX, testY, tokenizer, max_length)

In [27]:
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### 5) 모델 불러오기 및 설정

In [250]:
from transformers import BertForSequenceClassification 

num_labels = 2

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels
)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

### 6) 학습

In [251]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [252]:
model = model.to(device)

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_

# 하이퍼파라미터 설정
epochs = 5
optimizer = AdamW(model.parameters(), lr=1e-5)

# 스케줄러 설정
# 워밍업 단계(학습률을 선형적으로 증가) / 학습 단계(학습률을 선형적으로 감소)
total_steps = len(train_loader) * epochs 
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_history = {"train": [], "val": []}
best_loss_val = float('inf')
patience = 5
patience_cnt = 0

for epoch in range(epochs):
    #### Train #### 
    model.train()

    loss_train = 0.0
    for batch in train_loader:
        # GPU 보내기
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["label"].to(device)

        # 학습과정
        optimizer.zero_grad()
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = labels
        )
        loss = outputs.loss 
        loss.backward()
        # 기울기 폭주(exploration)
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        # loss 저장
        loss_train += loss.item() * batch_size 
    
    loss_history["train"].append(loss_train / len(train_dataset))

    #### Validation ####
    model.eval()

    loss_val = 0.0 
    with torch.no_grad():
        for batch in val_loader:
            # GPU 보내기
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["label"].to(device)

            # 예측
            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                token_type_ids = token_type_ids,
                labels = labels
            )
            loss = outputs.loss 

            # loss 저장 
            loss_val += loss.item() * batch_size

        loss_history["val"].append(loss_val / len(val_dataset))
    
    #### Early Stopping #### 
    if loss_val < best_loss_val:
        best_loss_val = loss_val 
        torch.save(model.state_dict(), "best_bert.pth")
    else:
        patience_cnt += 1
        if patience_cnt == patience:
            print("Early Stopping!")
            break 

    if epoch % 1 == 0:
        print(f"Epoch: {epoch} Train Loss: {loss_train / len(train_dataset)} Validation Loss: {loss_val / len(val_dataset)}")


### 7) 예측

In [34]:
# 모델 구조 가져오기
from transformers import BertForSequenceClassification 

num_labels = 2

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels
)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [35]:
import torch 
model.load_state_dict(torch.load("best_bert.pth"))

<All keys matched successfully>

In [36]:
model = model.to(device)

#### 성능평가

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [38]:
#### test ###
model.eval()

loss_test = 0.0 
correct = 0
with torch.no_grad():
    for batch in test_loader:
        # GPU 보내기
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["label"].to(device)

        # 예측
        outputs = model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            labels = labels
        )
        
        loss = outputs.loss 
        preds = outputs.logits.argmax(dim=1)
        correct += (labels == preds).sum()

        # loss 저장 
        loss_test += loss.item() * batch_size

print(f"Test Loss: {loss_test / len(test_dataset)} Accuracy: {correct / len(test_dataset)}")

Test Loss: 0.5207462197542191 Accuracy: 0.7699999809265137


#### 새로운 데이터 예측

In [39]:
# 모델 구조 가져오기
from transformers import BertForSequenceClassification 

num_labels = 2

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=num_labels
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
# 가중치 입히기
import torch 
model.load_state_dict(torch.load("best_bert.pth"))

<All keys matched successfully>

In [44]:
model = model.to(device)

In [48]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
max_length = 128

text = "배달도 늦고 맛도 없어요"

encoded_input = tokenizer(
    text, 
    max_length = max_length,
    padding="max_length",
    truncation=True,
    add_special_tokens=True,
    return_token_type_ids=True,
    return_attention_mask=True,
    return_tensors='pt'
)
# print(encoded_input)
encoded_input = encoded_input.to(device)
output = model(**encoded_input)
print(output.logits.argmax(dim=1).cpu().item())

0
